In [1]:
import pandas as pd

# Load your data
df = pd.read_csv('ESc1_filtered.csv')

# Assume these columns: Time, Type, BidPrice, BidSize, AskPrice, AskSize
# Adjust the column names based on your file!

# Initialize
states = []
actions = []
next_states = []

# Helper function to construct a state
def construct_state(row):
    return (row['Bid Price'], row['Bid Size'], row['Ask Price'], row['Ask Size'])

# Iterate over the rows
for i in range(len(df) - 1):
    current_row = df.iloc[i]
    next_row = df.iloc[i+1]

    # Build current state
    state = construct_state(current_row)

    # Infer action based on "Type" and Trade Price vs Bid/Ask
    if current_row['Type'] == 'Trade':
        trade_price = current_row['Bid Price']  # or get the TradePrice column if separate
        
        if abs(trade_price - current_row['Ask Price']) < 1e-4:
            action = 'BUY'
        elif abs(trade_price - current_row['Bid Price']) < 1e-4:
            action = 'SELL'
        else:
            action = 'WAIT'
    else:
        action = 'WAIT'  # If Quote update only → Assume WAIT

    # Build next state
    next_state = construct_state(next_row)

    # Save to lists
    states.append(state)
    actions.append(action)
    next_states.append(next_state)

# Now you have the full MDP dataset
mdp = pd.DataFrame({
    'State': states,
    'Action': actions,
    'NextState': next_states
})

print(mdp.head())

# Save for future use
mdp.to_csv('mdp_orderbook.csv', index=False)


                           State Action                      NextState
0  (4378.5, 30.0, 4378.75, 24.0)   WAIT  (4378.5, 30.0, 4378.75, 23.0)
1  (4378.5, 30.0, 4378.75, 23.0)   WAIT  (4378.5, 31.0, 4378.75, 23.0)
2  (4378.5, 31.0, 4378.75, 23.0)   WAIT  (4378.5, 32.0, 4378.75, 23.0)
3  (4378.5, 32.0, 4378.75, 23.0)   WAIT  (4378.5, 32.0, 4378.75, 22.0)
4  (4378.5, 32.0, 4378.75, 22.0)   WAIT  (4378.5, 32.0, 4378.75, 19.0)


In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process.kernels import RBF
import re

# Load the processed MDP data
mdp = pd.read_csv('mdp_orderbook.csv')

# Convert state tuples from string representation to actual tuples
def parse_state(state_str):
    # Remove outer parentheses and split by comma
    values = state_str.strip('()').split(',')
    # Convert to appropriate numeric types
    parsed_values = []
    
    for v in values:
        v = v.strip()
        # Check if value contains numpy function call
        if 'np.' in v:
            # Extract just the numeric value, properly handling parentheses
            match = re.search(r'\((.*?)(\)|$)', v)  # Modified regex to handle closing parenthesis
            if match:
                num_str = match.group(1).strip()
                parsed_values.append(float(num_str))
            else:
                # If regex doesn't match, try a different approach
                try:
                    # Remove any non-numeric chars except decimal point and negative sign
                    clean_v = re.sub(r'[^\d.-]', '', v)
                    parsed_values.append(float(clean_v))
                except ValueError:
                    print(f"Warning: Could not parse value: {v}")
                    parsed_values.append(np.nan)
        else:
            try:
                parsed_values.append(float(v))
            except ValueError:
                # Handle potential extra parentheses or other characters
                clean_v = re.sub(r'[^\d.-]', '', v)
                try:
                    parsed_values.append(float(clean_v))
                except ValueError:
                    print(f"Warning: Could not parse value: {v}")
                    parsed_values.append(np.nan)
    
    return tuple(parsed_values)
    
    return tuple(parsed_values)
# Apply parsing if states are stored as strings
if isinstance(mdp['State'].iloc[0], str):
    mdp['State'] = mdp['State'].apply(parse_state)
    mdp['NextState'] = mdp['NextState'].apply(parse_state)

# Convert states to feature vectors for ML algorithms
def state_to_features(state):
    # Extract bid-ask spread, sizes, and other features
    bid_price, bid_size, ask_price, ask_size = state
    spread = ask_price - bid_price
    imbalance = bid_size / (bid_size + ask_size) if (bid_size + ask_size) > 0 else 0.5
    
    return np.array([bid_price, bid_size, ask_price, ask_size, spread, imbalance])

# Prepare data
X = np.vstack([state_to_features(state) for state in mdp['State']])
y = mdp['Action']

# Encode actions as integers
action_map = {'WAIT': 0, 'BUY': 1, 'SELL': 2}
y_encoded = np.array([action_map[a] for a in y])

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ====== LNIRL Implementation ======
def linear_irl(states, actions, gamma=0.9, learning_rate=0.01, iterations=1000):
    """
    Linear IRL implementation to recover the reward function
    
    Parameters:
    states - array of state features
    actions - array of corresponding actions
    gamma - discount factor
    learning_rate - step size for gradient ascent
    iterations - number of iterations for optimization
    
    Returns:
    weights - learned feature weights for the reward function
    """
    num_features = states.shape[1]
    weights = np.random.rand(num_features)
    
    # Get unique actions
    unique_actions = np.unique(actions)
    num_actions = len(unique_actions)
    
    for _ in range(iterations):
        # Calculate rewards
        rewards = np.dot(states, weights)
        
        # Compute gradients for each action
        gradients = np.zeros_like(weights)
        
        for action in unique_actions:
            # Expert demonstrations for this action
            expert_states = states[actions == action]
            
            if len(expert_states) > 0:
                # Compute expert feature expectations
                expert_features = np.mean(expert_states, axis=0)
                
                # Compute features for all states where this action was taken
                action_probability = len(expert_states) / len(states)
                
                # Update gradient
                gradients += action_probability * (expert_features - np.mean(states, axis=0))
        
        # Update weights
        weights += learning_rate * gradients
        
        # Normalize weights
        weights = weights / np.linalg.norm(weights)
        
    return weights

# Apply LNIRL to learn reward weights
lnirl_weights = linear_irl(X_train_scaled, y_train)
print("LNIRL Learned Weights:", lnirl_weights)

# Calculate rewards using learned weights
lnirl_rewards = np.dot(X_test_scaled, lnirl_weights)

# ====== GPIRL Implementation ======
def gaussian_process_irl(states, actions, gamma=0.9, sigma=1.0, iterations=100):
    """
    Gaussian Process IRL implementation
    
    Parameters:
    states - array of state features
    actions - array of corresponding actions
    gamma - discount factor
    sigma - kernel bandwidth parameter
    iterations - number of iterations
    
    Returns:
    alpha - GP weights
    """
    n_samples = states.shape[0]
    
    # Compute kernel matrix
    kernel = RBF(length_scale=sigma)
    K = kernel(states)
    
    # Initialize alpha (GP weights)
    alpha = np.zeros(n_samples)
    
    for _ in range(iterations):
        # Compute rewards
        rewards = K @ alpha
        
        # Compute gradient of max likelihood
        gradient = np.zeros(n_samples)
        
        # For each action, compute the likelihood gradient
        unique_actions = np.unique(actions)
        for action in unique_actions:
            action_mask = actions == action
            if np.any(action_mask):
                # Expert feature expectations
                expert_indices = np.where(action_mask)[0]
                non_expert_indices = np.where(~action_mask)[0]
                
                if len(expert_indices) > 0 and len(non_expert_indices) > 0:
                    # Compute likelihood ratio gradient
                    expert_rewards = rewards[expert_indices]
                    non_expert_rewards = rewards[non_expert_indices]
                    
                    # Calculate softmax probabilities
                    max_reward = np.max(rewards)
                    exp_rewards = np.exp(rewards - max_reward)
                    Z = np.sum(exp_rewards)
                    
                    # Update gradient
                    for idx in expert_indices:
                        gradient[idx] += 1.0 / len(expert_indices) - exp_rewards[idx] / Z
        
        # Update alpha
        alpha += 0.01 * gradient
        
    return alpha

# Apply GPIRL
gpirl_weights = gaussian_process_irl(X_train_scaled, y_train)
print("GPIRL Learned Weights shape:", gpirl_weights.shape)

# Compute GP rewards for test set
kernel = RBF(length_scale=1.0)
K_test = kernel(X_test_scaled, X_train_scaled)
gpirl_rewards = K_test @ gpirl_weights

# ====== SVM Training on Recovered Rewards ======
# Combine original features with recovered rewards
X_train_with_rewards = np.column_stack([X_train_scaled, np.dot(X_train_scaled, lnirl_weights)])
X_test_with_rewards = np.column_stack([X_test_scaled, np.dot(X_test_scaled, lnirl_weights)])

# Train SVM
svm = SVC(kernel='rbf', C=10.0)
svm.fit(X_train_with_rewards, y_train)

# Evaluate accuracy
y_pred = svm.predict(X_test_with_rewards)
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy with LNIRL rewards: {accuracy:.4f}")

# Try with GPIRL rewards
# Compute GP rewards for training set
K_train = kernel(X_train_scaled, X_train_scaled)
gpirl_train_rewards = K_train @ gpirl_weights

# Combine features with GP rewards
X_train_with_gp_rewards = np.column_stack([X_train_scaled, gpirl_train_rewards])
X_test_with_gp_rewards = np.column_stack([X_test_scaled, gpirl_rewards])

# Train SVM with GP rewards
svm_gp = SVC(kernel='rbf', C=10.0)
svm_gp.fit(X_train_with_gp_rewards, y_train)

# Evaluate accuracy
y_pred_gp = svm_gp.predict(X_test_with_gp_rewards)
accuracy_gp = accuracy_score(y_test, y_pred_gp)
print(f"SVM Accuracy with GPIRL rewards: {accuracy_gp:.4f}")

# ====== Multiple Sampling Rounds Evaluation ======
def evaluate_multiple_rounds(X, y, n_rounds=5, test_size=0.3):
    lnirl_accuracies = []
    gpirl_accuracies = []
    baseline_accuracies = []
    
    for i in range(n_rounds):
        print(f"Evaluation round {i+1}/{n_rounds}")
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i*10)
        
        # Standardize
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # LNIRL
        lnirl_weights = linear_irl(X_train_scaled, y_train, iterations=500)
        X_train_lnirl = np.column_stack([X_train_scaled, np.dot(X_train_scaled, lnirl_weights)])
        X_test_lnirl = np.column_stack([X_test_scaled, np.dot(X_test_scaled, lnirl_weights)])
        
        # GPIRL
        gpirl_weights = gaussian_process_irl(X_train_scaled, y_train, iterations=50)
        kernel = RBF(length_scale=1.0)
        K_train = kernel(X_train_scaled, X_train_scaled)
        K_test = kernel(X_test_scaled, X_train_scaled)
        gpirl_train_rewards = K_train @ gpirl_weights
        gpirl_test_rewards = K_test @ gpirl_weights
        X_train_gpirl = np.column_stack([X_train_scaled, gpirl_train_rewards])
        X_test_gpirl = np.column_stack([X_test_scaled, gpirl_test_rewards])
        
        # Train models
        svm_baseline = SVC(kernel='rbf', C=10.0)
        svm_baseline.fit(X_train_scaled, y_train)
        
        svm_lnirl = SVC(kernel='rbf', C=10.0)
        svm_lnirl.fit(X_train_lnirl, y_train)
        
        svm_gpirl = SVC(kernel='rbf', C=10.0)
        svm_gpirl.fit(X_train_gpirl, y_train)
        
        # Evaluate
        baseline_acc = accuracy_score(y_test, svm_baseline.predict(X_test_scaled))
        lnirl_acc = accuracy_score(y_test, svm_lnirl.predict(X_test_lnirl))
        gpirl_acc = accuracy_score(y_test, svm_gpirl.predict(X_test_gpirl))
        
        baseline_accuracies.append(baseline_acc)
        lnirl_accuracies.append(lnirl_acc)
        gpirl_accuracies.append(gpirl_acc)
        
        print(f"  Baseline: {baseline_acc:.4f}, LNIRL: {lnirl_acc:.4f}, GPIRL: {gpirl_acc:.4f}")
    
    # Average results
    print("\nAverage Results:")
    print(f"Baseline: {np.mean(baseline_accuracies):.4f} ± {np.std(baseline_accuracies):.4f}")
    print(f"LNIRL: {np.mean(lnirl_accuracies):.4f} ± {np.std(lnirl_accuracies):.4f}")
    print(f"GPIRL: {np.mean(gpirl_accuracies):.4f} ± {np.std(gpirl_accuracies):.4f}")
    
    return {
        'baseline': baseline_accuracies,
        'lnirl': lnirl_accuracies,
        'gpirl': gpirl_accuracies
    }

# Run multiple evaluation rounds
results = evaluate_multiple_rounds(X, y_encoded, n_rounds=5)

# Visualize results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.boxplot([results['baseline'], results['lnirl'], results['gpirl']], 
            labels=['Baseline', 'LNIRL', 'GPIRL'])
plt.title('Classification Accuracy Comparison')
plt.ylabel('Accuracy')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig('irl_comparison_results.png')
plt.close()

print("Analysis complete! Results saved to 'irl_comparison_results.png'")

LNIRL Learned Weights: [0.44297942 0.12988212 0.04461005 0.62721423 0.48056566 0.40071034]
GPIRL Learned Weights shape: (33870,)
SVM Accuracy with LNIRL rewards: 0.9426
SVM Accuracy with GPIRL rewards: 0.9426
Evaluation round 1/5
  Baseline: 0.9426, LNIRL: 0.9426, GPIRL: 0.9426
Evaluation round 2/5
  Baseline: 0.9424, LNIRL: 0.9424, GPIRL: 0.9424
Evaluation round 3/5
  Baseline: 0.9441, LNIRL: 0.9441, GPIRL: 0.9441
Evaluation round 4/5
  Baseline: 0.9428, LNIRL: 0.9428, GPIRL: 0.9428
Evaluation round 5/5
  Baseline: 0.9436, LNIRL: 0.9436, GPIRL: 0.9436

Average Results:
Baseline: 0.9431 ± 0.0007
LNIRL: 0.9431 ± 0.0007
GPIRL: 0.9431 ± 0.0007
Analysis complete! Results saved to 'irl_comparison_results.png'


/tmp/ipykernel_556755/2403632930.py:313: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([results['baseline'], results['lnirl'], results['gpirl']],
